In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search,
)

In [3]:
output_dir = "therapist_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-08-21 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-21 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-21 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-21 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-21 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# Baseline: FFN using signatures

First, we dimension reduce these and then take signatures. We use the path signature as input to the FFN for classification.

We want to choose a dimension and signature depth such that the number of terms in the signature is _roughly_ 384 so that it is comparable to the number of features that we used for the previous baseline where we computed the mean of the history. Again, we are concatenating the features we obtain with the current utterance embedding.

In [7]:
num_epochs = 100
hidden_dim_sizes = [[64, 64], [128, 128], [256, 256], [512, 512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

## Using log signature

In [28]:
log_signature_dimensions_and_sig_depths = [(28, 2), (10, 3), (6, 4)]

In [29]:
import signatory

[
    signatory.logsignature_channels(channels, depth)
    for (channels, depth) in log_signature_dimensions_and_sig_depths
]

[406, 650, 385, 2860, 406]

### Using UMAP

In [10]:
(
    ffn_logsignature_umap_kfold,
    best_ffn_logsignature_umap_kfold,
    _,
    __,
) = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="main_therapist_behaviour",
    embeddings=sbert_embeddings,
    y_data=y_data_therapist,
    output_dim=output_dim_therapist,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["umap"],
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths,
    path_indices=therapist_index,
    split_ids=therapist_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_umap_focal_{gamma}_kfold.csv",
    verbose=False,
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in therapist_talk_type_output/ffn_logsignature_umap_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in therapist_talk_type_output/ffn_logsignature_umap_focal_2_kfold_best_model.csv


In [11]:
ffn_logsignature_umap_kfold.groupby(
    ["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_4145876/3138393360.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_logsignature_umap_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate                       
6         4         (64, 64)   0.1          0.0001         0.394118  0.352967   
                                            0.0005         0.409235  0.359750   
                                            0.0010         0.408412  0.360108   
                               0.5          0.0001         0.376534  0.315117   
                                            0.0005         0.379317  0.313235   
...                                                             ...       ...   
28        2         (512, 512) 0.1          0.0005         0.557938  0.528803   
                                            0.0010         0.550664  0.519710   
                               0.5          0.0001         0.572549  0.540230   
                                            0.0005         0.564643  0.529407   
                                            0.0010         0.566287  0.533995   

                                                           precision  \
dimension sig_depth hidden_dim dropout_rate learning_rate              
6         4         (64, 64)   0.1          0.0001          0.357613   
                                            0.0005          0.366701   
                                            0.0010          0.369461   
                               0.5          0.0001          0.321768   
                                            0.0005          0.321062   
...                                                              ...   
28        2         (512, 512) 0.1          0.0005          0.535095   
                                            0.0010          0.523512   
                               0.5          0.0001          0.547240   
                                            0.0005          0.534169   
                                            0.0010          0.537370   

                                                             recall  \
dimension sig_depth hidden_dim dropout_rate learning_rate             
6         4         (64, 64)   0.1          0.0001         0.353381   
                                            0.0005         0.361748   
                                            0.0010         0.361218   
                               0.5          0.0001         0.325521   
                                            0.0005         0.325716   
...                                                             ...   
28        2         (512, 512) 0.1          0.0005         0.528186   
                                            0.0010         0.520820   
                               0.5          0.0001         0.539902   
                                            0.0005         0.531130   
                                            0.0010         0.533966   

                                                           valid_accuracy  \
dimension sig_depth hidden_dim dropout_rate learning_rate                   
6         4         (64, 64)   0.1          0.0001               0.517089   
                                            0.0005               0.514610   
                                            0.0010               0.503323   
                               0.5          0.0001               0.456487   
                                            0.0005               0.434810   
...                                                                   ...   
28        2         (512, 512) 0.1          0.0005               0.662922   
                                            0.0010               0.649525   
                               0.5          0.0001               0.675791   
                                            0.0005               0.667563   
                                            0.0010               0.651477   

                                                           valid_f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate             


In [12]:
best_ffn_logsignature_umap_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,loss_function,gamma,k_fold,n_splits,batch_size,input_dim,dimension,sig_depth,method,log_signature
0,None,0.562049,0.526296,"[0.5389885807504078, 0.3308714918759232, 0.483...",0.530003,"[0.5267857142857143, 0.32748538011695905, 0.46...",0.524420,"[0.5517702070808284, 0.33432835820895523, 0.50...",None,0.684019,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.567362,0.535584,"[0.5412754936872773, 0.36679841897233206, 0.49...",0.542706,"[0.5251256281407035, 0.3899159663865546, 0.469...",0.531942,"[0.5584502338009352, 0.34626865671641793, 0.53...",None,0.684177,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.570588,0.540950,"[0.5443668993020936, 0.37574850299401197, 0.50...",0.545919,"[0.5416666666666666, 0.3768768768768769, 0.475...",0.539010,"[0.5470941883767535, 0.3746268656716418, 0.546...",None,0.683386,...,focal,2,True,5,64,790,28,2,umap,True


In [13]:
best_ffn_logsignature_umap_kfold["f1"].mean()

0.5342767149369686

In [14]:
best_ffn_logsignature_umap_kfold["precision"].mean()

0.5395427457257204

In [15]:
best_ffn_logsignature_umap_kfold["recall"].mean()

0.5317906700088327

In [16]:
np.stack(best_ffn_logsignature_umap_kfold["f1_scores"]).mean(axis=0)

array([0.54154366, 0.35780614, 0.49703473, 0.74072234])

In [17]:
np.stack(best_ffn_logsignature_umap_kfold["precision_scores"]).mean(axis=0)

array([0.53119267, 0.36475941, 0.47029819, 0.79192072])

In [18]:
np.stack(best_ffn_logsignature_umap_kfold["recall_scores"]).mean(axis=0)

array([0.55243821, 0.35174129, 0.52722063, 0.69576255])

### Using random projections

In [19]:
(
    ffn_logsignature_grp_kfold,
    best_ffn_logsignature_grp_kfold,
    _,
    __,
) = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="main_therapist_behaviour",
    embeddings=sbert_embeddings,
    y_data=y_data_therapist,
    output_dim=output_dim_therapist,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["gaussian_random_projection"],
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths,
    path_indices=therapist_index,
    split_ids=therapist_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_grp_focal_{gamma}_kfold.csv",
    verbose=False,
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in therapist_talk_type_output/ffn_logsignature_grp_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in therapist_talk_type_output/ffn_logsignature_grp_focal_2_kfold_best_model.csv


In [20]:
ffn_logsignature_grp_kfold.groupby(
    ["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_4145876/2536377135.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_logsignature_grp_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate                       
6         4         (64, 64)   0.1          0.0001         0.661037  0.622556   
                                            0.0005         0.661860  0.624669   
                                            0.0010         0.660911  0.624057   
                               0.5          0.0001         0.671411  0.632531   
                                            0.0005         0.669513  0.629569   
...                                                             ...       ...   
28        2         (512, 512) 0.1          0.0005         0.659836  0.621421   
                                            0.0010         0.650980  0.612874   
                               0.5          0.0001         0.673751  0.634017   
                                            0.0005         0.660342  0.621898   
                                            0.0010         0.664959  0.625067   

                                                           precision  \
dimension sig_depth hidden_dim dropout_rate learning_rate              
6         4         (64, 64)   0.1          0.0001          0.624165   
                                            0.0005          0.625164   
                                            0.0010          0.624860   
                               0.5          0.0001          0.635182   
                                            0.0005          0.631541   
...                                                              ...   
28        2         (512, 512) 0.1          0.0005          0.620673   
                                            0.0010          0.612643   
                               0.5          0.0001          0.633486   
                                            0.0005          0.621006   
                                            0.0010          0.625665   

                                                             recall  \
dimension sig_depth hidden_dim dropout_rate learning_rate             
6         4         (64, 64)   0.1          0.0001         0.622664   
                                            0.0005         0.625779   
                                            0.0010         0.625429   
                               0.5          0.0001         0.631329   
                                            0.0005         0.628694   
...                                                             ...   
28        2         (512, 512) 0.1          0.0005         0.625297   
                                            0.0010         0.617838   
                               0.5          0.0001         0.634746   
                                            0.0005         0.625403   
                                            0.0010         0.629198   

                                                           valid_accuracy  \
dimension sig_depth hidden_dim dropout_rate learning_rate                   
6         4         (64, 64)   0.1          0.0001               0.751846   
                                            0.0005               0.748049   
                                            0.0010               0.739241   
                               0.5          0.0001               0.752690   
                                            0.0005               0.750264   
...                                                                   ...   
28        2         (512, 512) 0.1          0.0005               0.742405   
                                            0.0010               0.730538   
                               0.5          0.0001               0.755169   
                                            0.0005               0.745833   
                                            0.0010               0.739504   

                                                           valid_f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate             


In [21]:
best_ffn_logsignature_grp_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,loss_function,gamma,k_fold,n_splits,batch_size,input_dim,dimension,sig_depth,method,log_signature
0,None,0.670588,0.631464,"[0.7027027027027026, 0.43145743145743143, 0.58...",0.630492,"[0.69355888093689, 0.4175977653631285, 0.6, 0....",0.632995,"[0.7120908483633934, 0.4462686567164179, 0.565...",None,0.759652,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.669260,0.629493,"[0.7118193891102258, 0.41980474198047424, 0.57...",0.628243,"[0.7075907590759076, 0.39397905759162305, 0.59...",0.632057,"[0.7160988643954576, 0.44925373134328356, 0.55...",None,0.757911,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.669639,0.630071,"[0.7056899934597776, 0.43595505617977537, 0.57...",0.630008,"[0.6912235746316464, 0.43759398496240604, 0.58...",0.630456,"[0.7207748830995324, 0.4343283582089552, 0.559...",None,0.760918,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True


In [22]:
best_ffn_logsignature_grp_kfold["f1"].mean()

0.6303426787277596

In [23]:
best_ffn_logsignature_grp_kfold["precision"].mean()

0.6295808347631123

In [24]:
best_ffn_logsignature_grp_kfold["recall"].mean()

0.6318361956364839

In [25]:
np.stack(best_ffn_logsignature_grp_kfold["f1_scores"]).mean(axis=0)

array([0.70673736, 0.42907241, 0.57656242, 0.80899852])

In [26]:
np.stack(best_ffn_logsignature_grp_kfold["precision_scores"]).mean(axis=0)

array([0.69745774, 0.41639027, 0.59451287, 0.80996246])

In [27]:
np.stack(best_ffn_logsignature_grp_kfold["recall_scores"]).mean(axis=0)

array([0.71632153, 0.44328358, 0.55969436, 0.8080453 ])